# Table of Contents
 <p>

In [4]:
CMDSTAN_HOME

"/n/home04/mrischard/cmdstan-2.12.0/"

In [5]:
using Stan
using DataFrames
using GaussianProcesses
using Proj4
using PDMats: PDMat
using DataFrames: head
using Base.Dates: Day, Hour
;

using JLD

using GaussianProcesses: SumKernel

Environment variable JULIA_SVG_BROWSER not found.


In [6]:
include("src/utils.jl")
include("src/preprocessing.jl")
include("src/variogram.jl")

LoadError: LoadError: LoadError: error in method definition: function Base.cov must be explicitly imported to be extended
while loading /n/home04/mrischard/temperature_model/src/variogram.jl, in expression starting on line 19
while loading In[6], in expression starting on line 3

In [7]:
isdList=read_isdList()
isdSubset=isdList[[(usaf in (725450,725460,725480,725485)) for usaf in isdList[:USAF].values],:]
isdSubset

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,725450,14990,THE EASTERN IOWA AIRPORT,US,IA,KCID,41.883,-91.717,264.6,1973,2015,1.64799e6,1.0441e6
2,725460,14933,DES MOINES INTERNATIONAL AIRPORT,US,IA,KDSM,41.534,-93.653,291.7,1973,2015,1.48723e6,1.00379e6
3,725480,94910,WATERLOO MUNICIPAL AIRPORT,US,IA,KALO,42.554,-92.401,264.6,1960,2015,1.59025e6,1.11766e6
4,725485,14940,MASON CITY MUNICIPAL ARPT,US,IA,KMCW,43.154,-93.327,373.4,1973,2015,1.51407e6,1.18374e6


In [8]:
hourly_cat=read_Stations(isdSubset)
itest=3

3

In [9]:
TnTx = test_data(hourly_cat, itest, Hour(17))

,ts_day,Tn,Tx,times_p_day
1,2015-01-01,-11.1,-3.9,20
2,2015-01-02,-10.6,1.7,30
3,2015-01-03,-5.6,-1.0,43
4,2015-01-04,-15.0,0.6,62
5,2015-01-05,-23.3,-13.9,28
6,2015-01-06,-27.8,-13.9,47
7,2015-01-07,-21.1,-13.3,32
8,2015-01-08,-24.4,-14.4,39
9,2015-01-09,-17.8,-6.1,77
10,2015-01-10,-21.1,-11.1,29


In [10]:
module pred
    using PDMats: PDMat
    using DataFrames
    using GaussianProcesses: GP, Kernel, MeanZero, predict
    using Base.Dates: Day, Hour
    using Stan
    using DataFrames: DataFrame, by

    include("src/utils.jl")
    include("src/predict_from_nearby.jl")
    include("src/stan_impute.jl")
end

pred

In [11]:
nearby_pred=load("saved/predictions_from_nearby/725480_2015-01-01_to_2015-03-14.jld")["nearby_pred"]

pred.NearbyPrediction(DateTime[2015-01-01T00:00:00,2015-01-01T00:54:00,2015-01-01T01:54:00,2015-01-01T02:54:00,2015-01-01T03:54:00,2015-01-01T04:54:00,2015-01-01T05:54:00,2015-01-01T06:00:00,2015-01-01T06:54:00,2015-01-01T07:54:00  …  2015-03-14T15:54:00,2015-03-14T16:54:00,2015-03-14T17:54:00,2015-03-14T18:00:00,2015-03-14T18:54:00,2015-03-14T19:54:00,2015-03-14T20:54:00,2015-03-14T21:54:00,2015-03-14T22:54:00,2015-03-14T23:54:00],[-1.36302,-2.03424,-2.4972,-2.78961,-3.2936,-3.51268,-3.79265,-3.7996,-3.79836,-3.83079  …  13.2922,15.8366,18.0617,18.2206,19.3765,20.5769,20.9216,20.8476,19.7075,16.5303],PDMats.PDMat{Float64,Array{Float64,2}}(2524,[26.6417 25.9274 … 6.45282 6.75446; 25.9274 26.378 … 5.81845 6.50425; … ; 6.45282 5.81845 … 26.3774 25.8338; 6.75446 6.50425 … 25.8338 26.3789],Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[5.16156 5.02317 … 1.25017 1.30861; 0.0 1.07041 … -0.431018 -0.0645656; … ; 0.0 0.0 … 0.723859 0.891823; 0.0 0.0 … 0.0 0.724852]))

In [12]:
imputation_data=pred.prep_data(nearby_pred, TnTx, Date(2015,3,1), Hour(17), Day(5))

Dict{String,Any} with 10 entries:
  "Tx"                 => [-4.4,-0.6,0.0,-9.4,-3.3]
  "predicted_cov_chol" => [5.13586 5.13416 … 1.31182 1.44402; 0.0 0.163135 … -0…
  "impt_times_p_day"   => [33,37,34,28,28]
  "Nimpt"              => 160
  "Tn"                 => [-21.1,-6.7,-14.4,-22.8,-17.8]
  "N_TxTn"             => 5
  "day_impute"         => [1,1,1,1,1,1,1,1,1,1  …  5,5,5,5,5,5,5,5,5,5]
  "predicted_cov"      => [26.3771 26.3683 … 6.73733 7.4163; 26.3683 26.3862 … …
  "predicted_mean"     => [1.6293,1.71351,2.39762,2.74985,3.22292,3.33772,2.627…
  "k_softmax"          => 10.0

In [13]:
imputation_model = pred.get_imputation_model();

In [ ]:
for window_days in (4:9)
    println("=========")
    @printf("%d days\n", window_days)
    imputation_data=pred.prep_data(nearby_pred, TnTx, Date(2015,3,1), Hour(17), Day(window_days))
    @time sim1 = stan(
        imputation_model, 
        [imputation_data], 
        CmdStanDir=Stan.CMDSTAN_HOME, 
        summary=false, 
        diagnostics=false
        )
    println("=========")
end

4 days

make: `/n/home04/mrischard/temperature_model/tmp/imputation' is up to date.

Length of data array is not equal to nchains,
all chains will use the first data dictionary.

514.874798 seconds (4.25 M allocations: 100.706 GB, 1.95% gc time)
5 days

make: `/n/home04/mrischard/temperature_model/tmp/imputation' is up to date.

Length of data array is not equal to nchains,
all chains will use the first data dictionary.

